In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  #默认为'last'
%matplotlib inline
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import time
import os
import sys
import random
import re
from numpy import linalg
np.set_printoptions(threshold=np.inf, precision=6, suppress=True)

In [1]:
%whos

Variable        Type        Data/Info
-------------------------------------
json            module      <module 'json' from 'D:\\<...>\lib\\json\\__init__.py'>
yapf_reformat   function    <function yapf_reformat at 0x000002391814C438>


[Jupyter Notebook 有哪些奇技淫巧？](https://www.zhihu.com/question/266988943)

[Anaconda下的Jupyter Notebook中使用Cython](https://blog.csdn.net/dzhanghz/article/details/104403654)  

In [3]:
%load_ext cython

In [8]:
%%cython
def add_c(a, b):
    for i in range(100000):
        c = a + b
    return c

## cython

## sys.modules

In [9]:
sys.modules # 所有模块

{'sys': <module 'sys' (built-in)>,
 'builtins': <module 'builtins' (built-in)>,
 '_frozen_importlib': <module 'importlib._bootstrap' (frozen)>,
 '_imp': <module '_imp' (built-in)>,
 '_thread': <module '_thread' (built-in)>,
 '_warnings': <module '_warnings' (built-in)>,
 '_weakref': <module '_weakref' (built-in)>,
 'zipimport': <module 'zipimport' (built-in)>,
 '_frozen_importlib_external': <module 'importlib._bootstrap_external' (frozen)>,
 '_io': <module 'io' (built-in)>,
 'marshal': <module 'marshal' (built-in)>,
 'nt': <module 'nt' (built-in)>,
 'winreg': <module 'winreg' (built-in)>,
 'encodings': <module 'encodings' from 'D:\\MySoft\\Anaconda3\\lib\\encodings\\__init__.py'>,
 'codecs': <module 'codecs' from 'D:\\MySoft\\Anaconda3\\lib\\codecs.py'>,
 '_codecs': <module '_codecs' (built-in)>,
 'encodings.aliases': <module 'encodings.aliases' from 'D:\\MySoft\\Anaconda3\\lib\\encodings\\aliases.py'>,
 'encodings.utf_8': <module 'encodings.utf_8' from 'D:\\MySoft\\Anaconda3\\lib\\enc

In [10]:
sys.modules[add_c.__module__] # cython模块信息

<module '_cython_magic_6c5f2df1aceb7efe8d5991e5b0499628' (C:\Users\wen\.ipython\cython\_cython_magic_6c5f2df1aceb7efe8d5991e5b0499628.cp37-win_amd64.pyd)>

In [12]:
add_c.__module__ # cython模块名

'_cython_magic_6c5f2df1aceb7efe8d5991e5b0499628'

### 用-n参数指定编译之后的扩展模块名

In [14]:
%%cython -n add_d
def add_d(a, b):
    for i in range(100000):
        c = a + b
    return c

### 用-a参数查看编译后的c语言程序

In [7]:
%%cython -a
a = 1.0
b = 2.0
c = a+b

### cdef

In [23]:
%%cython -a -n var1
cdef double a1 = 1.0
cdef double b1 = 2.0
cdef double c1 = a1+b1

In [24]:
a1
type(a1)

NameError: name 'a1' is not defined

In [22]:
%%cython -a
cdef double s = 0.0
a = 3.0
s = s + a
s = s + <double>a


In [25]:
%%cython -a
cdef list clist = [1,2,3]
cdef int cindex=0
clist[cindex]

plist = [1,2,3]

pindex=0
plist[pindex]

## loop

In [48]:
def add_python(a,b):
    for i in range(1000000):
        c = a + b
    return a + b

In [8]:
a = 1
b = 1

In [49]:
%%timeit
for i in range(1):
    add_c(a,b)

8.51 ms ± 99.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
%%timeit
for i in range(1):
    add_python(a,b)

44.9 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## fib

[Jupyter Notebook 有哪些奇技淫巧？](https://www.zhihu.com/question/266988943)  
可以使用单次运行的 ％time 或 ％timeit 来计算大量时间，并获得平均值和标准偏差。所以这对于这些简单的函数很有用，那么调用其他函数的函数呢？  
这里有 ％prun，我们创建了一个虚函数，能够调用 fib1（）很多时间。可以看到循环确实花了一些时间，但大部分时间花在 fib1（）上。


In [55]:
def fib1(n):
    a,b=0,1
    while b<n:
        a,b = b,a+b

In [56]:
%%cython
def fib2(n):
    a,b=0,1
    while b<n:
        a,b = b,a+b

In [68]:
%%cython
def fib3(int n): # 指定变量类型对加快速度非常有帮助
    cdef int a = 0
    cdef int b = 1
    cdef int t = 0
    while b<n:
        t = a
        a = b
        b = t + b

In [69]:
%timeit fib1(500)
%timeit fib2(500)
%timeit fib3(500) 

779 ns ± 23.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
363 ns ± 7.03 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
41 ns ± 0.724 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [81]:
def dummy1(n):
    for i in range(10000000):
        fib1(n)

In [85]:
def dummy2(n):
    for i in range(10000000):
        fib2(n)

In [87]:
def dummy3(n):
    for i in range(10000000):
        fib3(n)

In [83]:
%prun dummy1(50)

In [88]:
%prun dummy2(50)

In [89]:
%prun dummy3(50)

$$ P(A \mid B) = \frac{P(B \mid A) \, P(A)}{P(B)} $$
$a|b$  
$a\mid b$

In [90]:
#Note that http urls will not be displayed. Only https are allowed inside the Iframe
from IPython.display import IFrame
IFrame('https://en.wikipedia.org/wiki/HTTPS', width=800, height=450)


In [91]:
from IPython.display import IFrame
IFrame('https://arxiv.org/pdf/1406.2661.pdf', width=800, height=450)